In [141]:
library(FactoMineR)
library(factoextra)#fviz_eig
library(psych)

library(ggplot2)
library(pheatmap)
library(RColorBrewer)

library(tidyverse)
library(reshape)#melt

library(reshape2)
library(repr)
library(plyr)
library(Rmisc)
library(extrafont)
library(ggthemes)

library(lmtest)

library(ggpubr)


# Define Model

In [121]:
#define the model name
models <- c('FAVEE_model','Wish_1976_model','Triandis_1968_model','Marwell_1970_model',
            'Osgood_1957_model','Fiske_1992_model','Clark_2011_model','Carpendale_2004_model',
            'Foa_2012_model','Bugental_2000_model','Goffman_1959_model','Hamilton_1981_model',
            'Burton_1975_model','Rands_1979_model','Weiss_1998_model')

# define the model
FAVEE_model <- c("Formality.and.Regulation","Activeness","Valence.Evaluation","Goods.Exchange","Equality")
Wish_1976_model <- c("Formality.and.Regulation","Equality","Valence.Evaluation","Activity.Intensity")
Triandis_1968_model <- c("Valence.Evaluation","Equality","Intimacy") 
Marwell_1970_model <- c("Visibility","Formality.and.Regulation","Intimacy")
Osgood_1957_model <- c("Valence.Evaluation","Equality","Activity.Intensity")
Fiske_1992_model <- c("Communal.Sharing","Equality","Strategic","Expected.Reciprocity")
Clark_2011_model <- c("Communal.Sharing","Strategic","Expected.Reciprocity")
Carpendale_2004_model <- c("Importance.for.individuals","Importance.for.society")
Foa_2012_model <- c("Concreteness","Uniqueness")
Bugental_2000_model <- c("Attachment","Affiliation.Coalition","Mating","Expected.Reciprocity","Equality")
Goffman_1959_model <- c("Valence.Evaluation","Affiliation.Coalition","Conflict","Negotiation","Coercion")
Hamilton_1981_model <- c("Valence.Evaluation","Equality")
Burton_1975_model <- c("Valence.Evaluation","Equality","Occupational")
Montgomery_1988_model <- c("Valence.Evaluation","Equality","Intimacy")
Rands_1979_model <- c("Formality.and.Regulation","Socioemotional")
Weiss_1998_model <- c("Attachment","Affiliation.Coalition")

# Model Compare Function

In [123]:
model_compare <- function(model1,model2){
    data_regression <- data[-which(names(data) %in% c(model1,model2))]
    
    AdjR_model1 <- c()
    AdjR_model2 <- c()
    BIC_model1 <- c()
    BIC_model2 <- c()
    R_squared_change_Fvalue <- c()
    coxtest_z <- c()
    
    for (trait in c(1:ncol(data_regression))) {
        # Extract each trait to be used as dependent variable in our modeling
        y = data_regression[,trait]
        
        # Model comparison in model 1
        f_model1 <- as.formula(paste("y ~ ", paste(model1, collapse = "+")))
        lm_model1 = lm(f_model1, data = data) # Modeling
        AdjR_model1 <- c(AdjR_model1, summary(lm_model1)$adj.r.squared) # Extract each model's index and append it to the dependent variables' lists
        BIC_model1 <- c(BIC_model1, BIC(lm_model1))
        
        # model comparison in model 2
        f_model2 <- as.formula(paste("y ~ ", paste(model2, collapse = "+")))
        lm_model2 = lm(f_model2, data = data) # Modeling
        AdjR_model2 <- c(AdjR_model2, summary(lm_model2)$adj.r.squared) # Extract each model's index and append it to the dependent variables' lists
        BIC_model2 <- c(BIC_model2, BIC(lm_model2))

        R_anova <- anova(lm_model1, lm_model2)$F[2]

        coxtest_z <- c(coxtest_z, coxtest(lm_model1,lm_model2)[2,3])

        if(!is.na(R_anova)){
            R_squared_change_Fvalue <- c(R_squared_change_Fvalue, R_anova)
        }else{
            R_squared_change_Fvalue <- c(R_squared_change_Fvalue, -anova(lm_model2, lm_model1)$F[2])
        }
        
    }
    
    R_squared_change_Fvalue <- ifelse(is.na(R_squared_change_Fvalue), 0, R_squared_change_Fvalue)
    
    res_model1 <- c(mean(AdjR_model1),mean(BIC_model1),mean(R_squared_change_Fvalue),mean(coxtest_z))
    res_model2 <- c(mean(AdjR_model2),mean(BIC_model2),mean(R_squared_change_Fvalue),mean(coxtest_z))
    res_list <- list(res_model1,res_model2)
    
    return(res_list)
}

# Do model comparison

In [124]:
region_list <- c('Australia', 'Brazil', 'Chile', 'CHN', 'Egypt', 'France', 'Germany', 'HK', 'India', 'Israel', 'Japan', 'Mexico', 'Portugal', 'Qatar', 'Russia', 'South_africa', 'Spain', 'UK', 'USA')
model_rank_regions <- data.frame(region=character(), Model=character(), Rank=numeric(), Coxtext_Z=numeric())

for(region_index in c(1:length(region_list))){
    
    region <- region_list[region_index]
    data <- read.csv(file =  paste0("cleaning_results/", region, "/", region, "_dim_rel_scaled.csv"), header = TRUE, stringsAsFactors = FALSE,row.names = 1)

    model_first_res <- data.frame(Model=character(), AdjR=numeric(), BIC=numeric())
    
    for(i in c(1:length(models))){

        data_regression <- data[-which(names(data) %in% get(c(models[i])))]

        AdjR_model <- c()
        BIC_model <- c()

        for (trait in c(1:ncol(data_regression))) {
            # Extract each trait to be used as dependent variable in our modeling
            y = data_regression[,trait]
            
            # Model comparison in model 1
            f_model <- as.formula(paste("y ~ ", paste(get(c(models[i])), collapse = "+")))
            lm_model = lm(f_model, data = data) # Modeling
            AdjR_model <- c(AdjR_model, summary(lm_model)$adj.r.squared) # Extract each model's index and append it to the dependent variables' lists
            BIC_model <- c(BIC_model, BIC(lm_model))  
        }

        model_first_res <- rbind(model_first_res, c(models[i], mean(AdjR_model), mean(BIC_model)))

    }

    colnames(model_first_res) <- c('Model', 'AdjR', 'BIC')

    model_temp_max <- model_first_res$Model[which.max(model_first_res[,'AdjR'])]
    models_list_temp <- models

    model_rank_regions <- rbind(model_rank_regions, c(region, model_temp_max, 1, NA))

    for(i in c(1:(length(models)-1))){
        models_list_temp <- models_list_temp[models_list_temp != model_temp_max]
        model_comparesion_res <- data.frame(Model=character(),AdjR=numeric(),BIC=numeric(),Group=numeric(),Model_Names=character())

        for(j in c(1:length(models_list_temp))){
            model_compare_list <- model_compare(get(model_temp_max), get(models_list_temp[j]))
            
            model_comparesion_res <- rbind(model_comparesion_res,c(model_temp_max, unlist(model_compare_list[1]),j,models_list_temp[j]))
            model_comparesion_res <- rbind(model_comparesion_res,c('Others_Model', unlist(model_compare_list[2]),j,models_list_temp[j]))
        }

        colnames(model_comparesion_res) <- c('Model', 'AdjR', 'BIC','R_squared_change_Fvalue','Coxtext_Z','Group','Model_Names')

        max_index_temp <- which.max(model_comparesion_res$AdjR[model_comparesion_res$Model == 'Others_Model'])
        model_temp_max <- model_comparesion_res[model_comparesion_res$Model == 'Others_Model',][max_index_temp,'Model_Names']
        Coxtext_Z_temp_max <- model_comparesion_res[model_comparesion_res$Model == 'Others_Model',][max_index_temp,'Coxtext_Z']

        model_rank_regions <- rbind(model_rank_regions, c(region, model_temp_max, i+1, Coxtext_Z_temp_max))
    }
}

colnames(model_rank_regions) <- c('region', 'Model', 'Rank', 'Coxtext_Z')

In [125]:
print(model_rank_regions)

          region                 Model Rank         Coxtext_Z
1      Australia           FAVEE_model    1              <NA>
2      Australia       Wish_1976_model    2 -10.4316272035168
3      Australia   Bugental_2000_model    3 -8.59561442010888
4      Australia     Burton_1975_model    4 -7.63224147624137
5      Australia   Triandis_1968_model    5 -7.68675784384296
6      Australia     Osgood_1957_model    6 -5.59586861316594
7      Australia      Fiske_1992_model    7 -9.61824161832998
8      Australia      Weiss_1998_model    8 -10.8109585395377
9      Australia    Marwell_1970_model    9 -9.31869736056812
10     Australia      Clark_2011_model   10 -6.91658656030149
11     Australia    Goffman_1959_model   11 -14.4911441443794
12     Australia      Rands_1979_model   12 -15.6059330317355
13     Australia   Hamilton_1981_model   13 -12.6537994880516
14     Australia Carpendale_2004_model   14 -11.6466371503196
15     Australia        Foa_2012_model   15 -16.9012471520179
16      

In [202]:
model_rank_regions$Rank <- as.numeric(model_rank_regions$Rank)

wilcox.test(model_rank_regions$Rank[model_rank_regions$Model == 'FAVEE_model'], model_rank_regions$Rank[model_rank_regions$Model == 'Bugental_2000_model'], paired = TRUE)
t.test(model_rank_regions$Rank[model_rank_regions$Model == 'FAVEE_model'], model_rank_regions$Rank[model_rank_regions$Model == 'Bugental_2000_model'], paired = TRUE)

wilcox.test(model_rank_regions$Rank[model_rank_regions$Model == 'FAVEE_model'], model_rank_regions$Rank[model_rank_regions$Model == 'Wish_1976_model'], paired = TRUE)
t.test(model_rank_regions$Rank[model_rank_regions$Model == 'FAVEE_model'], model_rank_regions$Rank[model_rank_regions$Model == 'Wish_1976_model'], paired = TRUE)

# Define a function to calculate the standard error.
stderr <- function(x) {
  return(sd(x)/sqrt(length(x)))
}

# Calculate the average ranking and standard error for each model.
avg_rank <- aggregate(Rank ~ Model, data = df, FUN = mean)
avg_rank$SE <- aggregate(Rank ~ Model, data = df, FUN = stderr)$Rank

# Sort by ranking.
avg_rank <- avg_rank[order(avg_rank$Rank),]

output_file <- "model_rank.png"
png(output_file, bg = "transparent", family = 'sans', units = 'in', width = 13, height = 20, res = 300)
# Plot a bar chart and add error bars.
ggplot(avg_rank, aes(y = reorder(Model, -Rank), x = Rank)) +  # Note that the x and y are swapped here, and the Model is reordered using reorder.
  # Add a bar chart with a single color.
  geom_bar(stat = 'identity', width = 0.8, alpha = 0.8, color = 'black', fill = "#D6D6D6", size = 2) +
  # Add bold error bars.
  geom_errorbarh(aes(xmin = Rank - SE, xmax = Rank + SE), height = 0.2, size = 3) + # Used geom_errorbarh instead of geom_errorbar.
  # Used the classic theme as a base and made some adjustments.
  theme_classic() +
  theme(axis.text = element_blank(),
        axis.text.x = element_text(size=34, face="bold"), # Bolded and adjusted the x-axis labels.
        axis.title.x = element_blank(),
        axis.title.y = element_blank(),
        axis.line.x = element_line(linetype = 1, color = "black", size = 3), # Bolded the x-axis line.
        axis.line = element_blank(),
        legend.position = 'none',
        strip.text = element_blank())+
  scale_x_continuous(breaks=seq(1,17,1))

# Turn off the graphics device.
dev.off()

Warning message in wilcox.test.default(model_rank_regions$Rank[model_rank_regions$Model == :
"cannot compute exact p-value with ties"



	Wilcoxon signed rank test with continuity correction

data:  model_rank_regions$Rank[model_rank_regions$Model == "FAVEE_model"] and model_rank_regions$Rank[model_rank_regions$Model == "Bugental_2000_model"]
V = 11, p-value = 0.0005642
alternative hypothesis: true location shift is not equal to 0



	Paired t-test

data:  model_rank_regions$Rank[model_rank_regions$Model == "FAVEE_model"] and model_rank_regions$Rank[model_rank_regions$Model == "Bugental_2000_model"]
t = -5.3445, df = 18, p-value = 4.432e-05
alternative hypothesis: true mean difference is not equal to 0
95 percent confidence interval:
 -1.9063500 -0.8304921
sample estimates:
mean difference 
      -1.368421 


Warning message in wilcox.test.default(model_rank_regions$Rank[model_rank_regions$Model == :
"cannot compute exact p-value with ties"



	Wilcoxon signed rank test with continuity correction

data:  model_rank_regions$Rank[model_rank_regions$Model == "FAVEE_model"] and model_rank_regions$Rank[model_rank_regions$Model == "Wish_1976_model"]
V = 0, p-value = 9.801e-05
alternative hypothesis: true location shift is not equal to 0



	Paired t-test

data:  model_rank_regions$Rank[model_rank_regions$Model == "FAVEE_model"] and model_rank_regions$Rank[model_rank_regions$Model == "Wish_1976_model"]
t = -7.2396, df = 18, p-value = 9.859e-07
alternative hypothesis: true mean difference is not equal to 0
95 percent confidence interval:
 -2.240872 -1.232813
sample estimates:
mean difference 
      -1.736842 


png 
  2

In [126]:
# Filter and count for each rank.
model_counts <- list()
for(i in 1:length(models)) {
  df_rank <- model_rank_regions[model_rank_regions$Rank == i,]
  model_counts[[i]] <- as.data.frame(table(df_rank$Model))
  names(model_counts[[i]]) <- c("Model", "Frequency")
}

# Plot a histogram.
for(i in 1:5) {
  a = ggplot(model_counts[[i]], aes(x=reorder(Model, -Frequency), y=Frequency, fill=Model)) +
    geom_bar(stat="identity") +
    scale_fill_brewer(palette = "Set3") +
    theme_minimal() +
    theme(axis.text.x = element_text(angle = 60, hjust = 1, size = 12, color = "darkblue"),
          axis.text.y = element_text(size = 12, color = "darkblue"),
          plot.title = element_text(size = 14, color = "darkred")) +
    labs(x = "Models", y = "Frequency", title = paste("Model Frequencies for Rank", i)) +
    guides(fill=FALSE)  # Remove the legend.
}

In [127]:
df <- model_rank_regions

# First, we convert Rank to a factor so it displays correctly in the plot.
df$Rank <- as.factor(df$Rank)

# Create a list of models.
model_list <- unique(df$Model)

# Calculate the maximum value and the interval of the y-axis.
y_max <- max(table(df$Rank))
y_breaks <- seq(0, y_max, by = 5)

# Plot a histogram for each model.
for(model in model_list){
  df_model <- df[df$Model == model,]
  model_counts <- as.data.frame(table(df_model$Rank))
  names(model_counts) <- c("Rank", "Frequency")
  
  a = ggplot(model_counts, aes(x=Rank, y=Frequency)) +
    geom_bar(stat="identity", fill="steelblue") +
    coord_cartesian(ylim = c(0, y_max)) +
    scale_y_continuous(breaks = y_breaks) +
    theme_minimal() +
    theme(axis.text.x = element_text(size = 12, color = "darkblue"),
          axis.text.y = element_text(size = 12, color = "darkblue"),
          plot.title = element_text(size = 14, color = "darkred")) +
    labs(x = "Rank", y = "Frequency", title = paste("Frequency of Ranks for Model:", model))
}

In [129]:
df

region,Model,Rank,Coxtext_Z
<chr>,<chr>,<dbl>,<chr>
Australia,FAVEE_model,1,NA
Australia,Wish_1976_model,2,-10.4316272035168
Australia,Bugental_2000_model,3,-8.59561442010888
Australia,Burton_1975_model,4,-7.63224147624137
Australia,Triandis_1968_model,5,-7.68675784384296
Australia,Osgood_1957_model,6,-5.59586861316594
Australia,Fiske_1992_model,7,-9.61824161832998
Australia,Weiss_1998_model,8,-10.8109585395377
Australia,Marwell_1970_model,9,-9.31869736056812
